In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import re

spark = SparkSession.builder.appName("Colab Spark UI").config("spark.ui.port", "4050").getOrCreate()
!pip install pyngrok
from pyngrok import ngrok

In [ ]:
# Start ngrok tunnel for port 4050 (Spark UI port)
ngrok.set_auth_token("<INSERT YOUR AUTH TOKEN>")
ngrok_tunnel = ngrok.connect(4050)
print("Spark UI is accessible at:", ngrok_tunnel.public_url)

Spark UI is accessible at: https://2ca0-35-185-0-169.ngrok-free.app


In [ ]:
#https://dashboard.ngrok.com/signup
#https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
!pip install gdown

In [ ]:
!gdown 1zHgvidy9FvhZvE68S0mXWkoF-hHMpiUL

Downloading...
From: https://drive.google.com/uc?id=1zHgvidy9FvhZvE68S0mXWkoF-hHMpiUL
To: /content/The Project Gutenberg eBook of Le Morte D’Arthur, Volume I (of II), by Thomas Malory.html
100% 964k/964k [00:00<00:00, 409MB/s]


In [ ]:
from bs4 import BeautifulSoup

with open('The Project Gutenberg eBook of Le Morte D’Arthur, Volume I (of II), by Thomas Malory.html', 'r') as f:
  html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
max_chars = 1000

chapters = soup.find_all('div', class_='chapter') #{"class": "chapter"}
complete_text = ""
cont = 0
for i,chapter in enumerate(chapters[2:-1]):
    proc_txt = chapter.get_text().replace("\n"," ")[:max_chars].strip()
    if "BOOK " == proc_txt[:5]: continue
    proc_txt = re.sub(" +"," ",re.split(r'\.', proc_txt, 1)[1]).strip()
    complete_text += proc_txt + "\n"
    cont+=1
print(cont)

238


In [ ]:
with open('complete_text.txt', 'w') as f:
  f.write(complete_text)

In [ ]:
sc = spark.sparkContext

In [ ]:
txt = sc.textFile('complete_text.txt') #spaces here won't make the program work

In [ ]:
txt.count()

238

In [ ]:
txt.take(2)

['First, How Uther Pendragon sent for the duke of Cornwall and Igraine his wife, and of their departing suddenly again. It befell in the days of Uther Pendragon, when he was king of all England, and so reigned, that there was a mighty duke in Cornwall that held war against him long time. And the duke was called the Duke of Tintagil. And so by means King Uther sent for this duke, charging him to bring his wife with him, for she was called a fair lady, and a passing wise, and her name was called Igraine. So when the duke and his wife were come unto the king, by the means of great lords they were accorded both. The king liked and loved this lady well, and he made them great cheer out of measure, and desired to have lain by her. But she was a passing good woman, and would not assent unto the king. And then she told the duke her husband, and said, I suppose that we were sent for that I should be dishonoured; wherefore, husband, I counsel you, that we depart from hence suddenly,',
 'How Uthe

In [ ]:
line = 'The Project Gutenberg eBook of Le Morte D’Arthur, Volume I (of II), by Thomas Malory'

In [ ]:
line.split(' ')

['The',
 'Project',
 'Gutenberg',
 'eBook',
 'of',
 'Le',
 'Morte',
 'D’Arthur,',
 'Volume',
 'I',
 '(of',
 'II),',
 'by',
 'Thomas',
 'Malory']

In [ ]:
def splitline(line):
    return line.split(' ')
#lambda line: line.split(' ')

In [ ]:
words = txt.flatMap(splitline) #map, flatMap

In [ ]:
words.count()

45274

In [ ]:
words.take(2)

['First,', 'How']

In [ ]:
word_tuples = words.map(lambda word: (word, 1))

In [ ]:
word_tuples.take(5)

[('First,', 1), ('How', 1), ('Uther', 1), ('Pendragon', 1), ('sent', 1)]

In [ ]:
word_count = word_tuples.reduceByKey(lambda this_word_counter, that_word_counter:
                                      this_word_counter + that_word_counter)

In [ ]:
word_count.count()

4723

In [ ]:
word_count.take(10)

[('Uther', 12),
 ('Pendragon', 5),
 ('sent', 51),
 ('of', 1144),
 ('Cornwall', 17),
 ('Igraine', 4),
 ('his', 625),
 ('departing', 2),
 ('suddenly', 5),
 ('It', 20)]

In [ ]:
word_count.takeOrdered(10, lambda pair: -pair[1]) #and, or, not, the, of, he, to, a, his, was

[('and', 2567),
 ('the', 1940),
 ('Sir', 1191),
 ('of', 1144),
 ('that', 988),
 ('he', 925),
 ('to', 784),
 ('a', 774),
 ('his', 625),
 ('was', 569)]

In [ ]:
word_count.filter(lambda x: not x[0] in ['the', 'and', 'not', 'of']).takeOrdered(20, lambda pair: -pair[1])

[('Sir', 1191),
 ('that', 988),
 ('he', 925),
 ('to', 784),
 ('a', 774),
 ('his', 625),
 ('was', 569),
 ('with', 489),
 ('for', 486),
 ('I', 483),
 ('him', 480),
 ('said', 477),
 ('King', 452),
 ('in', 438),
 ('And', 380),
 ('they', 338),
 ('there', 308),
 ('so', 306),
 ('came', 285),
 ('this', 266)]

### Execute Word Count in one app

In [ ]:
sc.textFile("complete_text.txt").\
flatMap(splitline).\
map(lambda word: (word, 1)  ).\
reduceByKey(lambda this_word_counter, that_word_counter:
                                      this_word_counter + that_word_counter).\
takeOrdered(40, lambda pair: -pair[1])

[('and', 2567),
 ('the', 1940),
 ('Sir', 1191),
 ('of', 1144),
 ('that', 988),
 ('he', 925),
 ('to', 784),
 ('a', 774),
 ('his', 625),
 ('was', 569),
 ('with', 489),
 ('for', 486),
 ('I', 483),
 ('him', 480),
 ('said', 477),
 ('King', 452),
 ('in', 438),
 ('And', 380),
 ('they', 338),
 ('there', 308),
 ('so', 306),
 ('came', 285),
 ('this', 266),
 ('ye', 265),
 ('knight', 257),
 ('she', 252),
 ('as', 251),
 ('all', 245),
 ('is', 240),
 ('Then', 227),
 ('How', 222),
 ('had', 211),
 ('have', 210),
 ('by', 209),
 ('unto', 204),
 ('be', 203),
 ('Tristram', 199),
 ('were', 198),
 ('great', 193),
 ('it', 190)]

### Alternative app

In [ ]:
def splitline_into_tuples(line):
    words = line.split(' ')
    pairs = []
    for word in words:
        pairs.append( (word, 1)  )
    return pairs

In [ ]:
sc.textFile("complete_text.txt").\
flatMap(splitline_into_tuples).\
reduceByKey(lambda this_word_counter, that_word_counter:
                                      this_word_counter + that_word_counter).\
takeOrdered(40, lambda pair: -pair[1])

[('and', 2567),
 ('the', 1940),
 ('Sir', 1191),
 ('of', 1144),
 ('that', 988),
 ('he', 925),
 ('to', 784),
 ('a', 774),
 ('his', 625),
 ('was', 569),
 ('with', 489),
 ('for', 486),
 ('I', 483),
 ('him', 480),
 ('said', 477),
 ('King', 452),
 ('in', 438),
 ('And', 380),
 ('they', 338),
 ('there', 308),
 ('so', 306),
 ('came', 285),
 ('this', 266),
 ('ye', 265),
 ('knight', 257),
 ('she', 252),
 ('as', 251),
 ('all', 245),
 ('is', 240),
 ('Then', 227),
 ('How', 222),
 ('had', 211),
 ('have', 210),
 ('by', 209),
 ('unto', 204),
 ('be', 203),
 ('Tristram', 199),
 ('were', 198),
 ('great', 193),
 ('it', 190)]

# Yet another app
 * removes stopwords
 * does not consider upper/lower case

In [ ]:
stopWords = {'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves'}

In [ ]:
def split_clean_tokenize(line):
    # ignore empty lines
    if len(line) < 1:
        return []

    # convert to lowercase
    lowCase_str = line.lower()

    # remove punctuation
    punc = '!"#&\()*+,-./:;<=>?@[\\]^_{|}~$%'
    for ch in punc:
        lowCase_str = lowCase_str.replace(ch,'')

    # tokenize non stop words
    tupples = []
    for word in lowCase_str.split(' '):
        if word not in stopWords:
            tupples.append( (word, 1) )

    return tupples

In [ ]:
sc.textFile("complete_text.txt").\
flatMap(split_clean_tokenize).\
reduceByKey(lambda this_word_counter, that_word_counter:
                                      this_word_counter + that_word_counter).\
takeOrdered(40, lambda tupple: -tupple[1])

[('sir', 1286),
 ('king', 620),
 ('said', 544),
 ('knight', 430),
 ('tristram', 303),
 ('came', 291),
 ('ye', 284),
 ('arthur', 254),
 ('knights', 235),
 ('unto', 207),
 ('great', 195),
 ('launcelot', 180),
 ('shall', 164),
 ('well', 152),
 ('lady', 146),
 ('made', 145),
 ('would', 127),
 ('damosel', 122),
 ('rode', 122),
 ('upon', 119),
 ('thou', 114),
 ('gawaine', 102),
 ('two', 98),
 ('saw', 96),
 ('many', 94),
 ('horse', 94),
 ('took', 93),
 ('man', 90),
 ('sword', 88),
 ('good', 82),
 ('fair', 82),
 ('merlin', 81),
 ('one', 80),
 ('castle', 78),
 ('might', 75),
 ('met', 73),
 ('la', 73),
 ('ever', 72),
 ('smote', 72),
 ('let', 70)]

### Separate per chapter

In [ ]:
import os
# create chapters folder
os.mkdir('chapters')

max_chars = 1000
cont = 0
chapters = soup.find_all('div', class_='chapter')
for i,chapter in enumerate(chapters[2:-1]):
    proc_txt = chapter.get_text().replace("\n"," ")[:max_chars].strip()
    if "BOOK " == proc_txt[:5]:
        book_num = proc_txt[5:-1]
        continue
    app = re.split(r'\.', proc_txt, 1)
    proc_txt = re.sub(" +"," ",app[1]).strip()
    with open(f'chapters/{book_num}_{str(app[0].split(" ")[-1])}.txt', 'w') as f:
        f.write(proc_txt)
    cont+=1
    if cont >10: break
print(cont)

11


In [ ]:
book_1_1 = sc.textFile(f"chapters/I_I.txt").\
                flatMap(split_clean_tokenize).\
                    map(lambda token: ( ("I_I", token[0]) , token[1]))

book_1_2 = sc.textFile(f"chapters/I_II.txt").\
                flatMap(split_clean_tokenize).\
                    map(lambda token: ( ("I_II", token[0]) , token[1]))

num_documents = 2

In [ ]:
allbooks = book_1_1.union(book_1_2)

In [ ]:
allbooks

UnionRDD[113] at union at NativeMethodAccessorImpl.java:0

In [ ]:
book_1_1.take(5)

[(('I_I', 'first'), 1),
 (('I_I', 'uther'), 1),
 (('I_I', 'pendragon'), 1),
 (('I_I', 'sent'), 1),
 (('I_I', 'duke'), 1)]

In [ ]:
book_1_2.take(5)

[(('I_II', 'uther'), 1),
 (('I_II', 'pendragon'), 1),
 (('I_II', 'made'), 1),
 (('I_II', 'war'), 1),
 (('I_II', 'duke'), 1)]

In [ ]:
allbooks.filter(lambda token: token[0][0] == "I_I").take(10)

[(('I_I', 'first'), 1),
 (('I_I', 'uther'), 1),
 (('I_I', 'pendragon'), 1),
 (('I_I', 'sent'), 1),
 (('I_I', 'duke'), 1),
 (('I_I', 'cornwall'), 1),
 (('I_I', 'igraine'), 1),
 (('I_I', 'wife'), 1),
 (('I_I', 'departing'), 1),
 (('I_I', 'suddenly'), 1)]

In [ ]:
allbooks.filter(lambda token: token[0][1] == "uther").take(10)

[(('I_I', 'uther'), 1),
 (('I_I', 'uther'), 1),
 (('I_I', 'uther'), 1),
 (('I_II', 'uther'), 1),
 (('I_II', 'uther'), 1),
 (('I_II', 'uther'), 1)]

In [ ]:
alltuples = allbooks.reduceByKey(lambda this_word_counter, that_word_counter: this_word_counter + that_word_counter)

In [ ]:
alltuples.take(10)

[(('I_I', 'pendragon'), 2),
 (('I_I', 'duke'), 7),
 (('I_I', 'reigned'), 1),
 (('I_I', 'held'), 1),
 (('I_I', 'war'), 1),
 (('I_I', 'called'), 3),
 (('I_I', 'tintagil'), 1),
 (('I_I', 'fair'), 1),
 (('I_I', 'come'), 1),
 (('I_I', 'unto'), 2)]

Alltupples -> Key: documentID + token, Value: TF(token) for this document

AnotherRDD -> Key: token, Value: number of documents it appears in.

In [ ]:
anotherRDD = allbooks.reduceByKey(lambda this_word_counter, that_word_counter: 1)

In [ ]:
anotherRDD.filter(lambda token: token[0][1] == "uther").take(10)

[(('I_I', 'uther'), 1), (('I_II', 'uther'), 1)]

In [ ]:
documentsRDD = anotherRDD.map(lambda token: (token[0][1], 1) ).\
reduceByKey(lambda this_word_counter, that_word_counter: this_word_counter + that_word_counter)

In [ ]:
documentsRDD.take(10)

[('uther', 2),
 ('sent', 1),
 ('departing', 1),
 ('befell', 1),
 ('days', 1),
 ('king', 2),
 ('england', 1),
 ('mighty', 1),
 ('long', 2),
 ('lady', 1)]

In [ ]:
ndRDD = documentsRDD.map(lambda token: (token[0], ('ND', token[1]) ) )

In [ ]:
ndRDD.take(10)

[('uther', ('ND', 2)),
 ('sent', ('ND', 1)),
 ('departing', ('ND', 1)),
 ('befell', ('ND', 1)),
 ('days', ('ND', 1)),
 ('king', ('ND', 2)),
 ('england', ('ND', 1)),
 ('mighty', ('ND', 1)),
 ('long', ('ND', 2)),
 ('lady', ('ND', 1))]

In [ ]:
tfRDD = alltuples.map(lambda token: (token[0][1], ('TF-' + str(token[0][0]), token[1]) ) )

In [ ]:
tfRDD.take(10)

[('pendragon', ('TF-I_I', 2)),
 ('duke', ('TF-I_I', 7)),
 ('reigned', ('TF-I_I', 1)),
 ('held', ('TF-I_I', 1)),
 ('war', ('TF-I_I', 1)),
 ('called', ('TF-I_I', 3)),
 ('tintagil', ('TF-I_I', 1)),
 ('fair', ('TF-I_I', 1)),
 ('come', ('TF-I_I', 1)),
 ('unto', ('TF-I_I', 2))]

In [ ]:
tfidfRDD = tfRDD.union(ndRDD)

In [ ]:
tfidfRDD.filter(lambda word: word[0] == 'uther').collect()

[('uther', ('TF-I_I', 3)), ('uther', ('TF-I_II', 3)), ('uther', ('ND', 2))]

# Compute TF-IDF for a word

TF-IDF(w) = N(frequency of w in a document) * log( total_number_of_documents = 4 / Number_of_Documents_Containing_word_w


TF-IDF(w) -- we filter the RDD based on 'w' -- we need to get as result all the information for computing this formula

```
tf_idf_rdd.filter('dracula').collect()
[('dracula', ('number of documents it appears', 1) ),
('dracula', ('TF-0', 2332) )]

tf_idf_rdd.filter('project').collect()
[('project', ('number of documents it appears', 4) ),
('project', ('TF-0', 93) ),
('project', ('TF-1', 33) ),
('project', ('TF-2', 45) ),
('project', ('TF-3', 50) )]


tf_idf_rdd.filter('project').groupByKey().mapValues(list).collect()

('project', [('number of documents it appears', 4), ('TF-0', 93), ('TF-1', 33), ('TF-2', 45), ('TF-3', 50)] )

```

In [ ]:
tfRDD = allbooks.reduceByKey(lambda this_word_counter, that_word_counter: this_word_counter + that_word_counter)\
.map(lambda token: (token[0][1], ('TF-' + str(token[0][0]), token[1]) ) )

In [ ]:
ndRDD = allbooks.reduceByKey(lambda this_word_counter, that_word_counter: 1).map(lambda token: (token[0][1], 1) )\
.reduceByKey(lambda this_word_counter, that_word_counter: this_word_counter + that_word_counter)\
.map(lambda token: (token[0], ('ND', token[1]) ) )

In [ ]:
tfidfRDD = tfRDD.union(ndRDD)

In [ ]:
tfidfRDD.filter(lambda word: word[0] == 'uther').collect()

[('uther', ('TF-I_I', 3)), ('uther', ('TF-I_II', 3)), ('uther', ('ND', 2))]

In [ ]:
merge_tuples = tfidfRDD.groupByKey().mapValues(list)

In [ ]:
merge_tuples.take(10)

[('mighty', [('TF-I_I', 1), ('ND', 1)]),
 ('lords', [('TF-I_I', 1), ('ND', 1)]),
 ('liked', [('TF-I_I', 1), ('ND', 1)]),
 ('good', [('TF-I_I', 1), ('ND', 1)]),
 ('would', [('TF-I_I', 1), ('ND', 1)]),
 ('dishonoured', [('TF-I_I', 1), ('ND', 1)]),
 ('wherefore', [('TF-I_I', 1), ('ND', 1)]),
 ('gat', [('TF-I_II', 1), ('ND', 1)]),
 ('came', [('TF-I_II', 1), ('ND', 1)]),
 ('ware', [('TF-I_II', 1), ('ND', 1)])]

In [ ]:
import math

In [ ]:
def compute_tfidf(tupple):
    word = tupple[0]
    allterms = tupple[1]
    dictterms = {}
    for term in allterms:
        dictterms[term[0]] = term[1]

    nd = dictterms['ND']
    del dictterms['ND']

    result = []
    for key, value in dictterms.items():
        tfidf = value * math.log10(num_documents / nd)
        result.append( ('TF-IDF-' + key.split('-')[1], tfidf) )

    return (word, result)

In [ ]:
compute_tfidf( ('arthur', [('TF-2', 2), ('ND', 1)]) )

('arthur', [('TF-IDF-2', 0.6020599913279624)])

In [ ]:
final_tuples = tfidfRDD.groupByKey().mapValues(list).map(compute_tfidf)

In [ ]:
final_tuples.take(10)

[('mighty', [('TF-IDF-I_I', 0.3010299956639812)]),
 ('lords', [('TF-IDF-I_I', 0.3010299956639812)]),
 ('liked', [('TF-IDF-I_I', 0.3010299956639812)]),
 ('good', [('TF-IDF-I_I', 0.3010299956639812)]),
 ('would', [('TF-IDF-I_I', 0.3010299956639812)]),
 ('dishonoured', [('TF-IDF-I_I', 0.3010299956639812)]),
 ('wherefore', [('TF-IDF-I_I', 0.3010299956639812)]),
 ('gat', [('TF-IDF-I_II', 0.3010299956639812)]),
 ('came', [('TF-IDF-I_II', 0.3010299956639812)]),
 ('ware', [('TF-IDF-I_II', 0.3010299956639812)])]

In [ ]:
final_tuples.filter(lambda token: token[0] == 'uther').collect()

[('uther', [('TF-IDF-I_I', 0.0), ('TF-IDF-I_II', 0.0)])]

In [ ]:
final_tuples.filter(lambda token: token[0] == 'arthur').collect()

[('arthur', [('TF-IDF-I_II', 0.3010299956639812)])]

In [ ]:
ndRDD.filter(lambda token: token[1][1] == 2).take(10)

[('uther', ('ND', 2)),
 ('king', ('ND', 2)),
 ('long', ('ND', 2)),
 ('cornwall', ('ND', 2)),
 ('first', ('ND', 2)),
 ('igraine', ('ND', 2)),
 ('told', ('ND', 2)),
 ('said', ('ND', 2)),
 ('pendragon', ('ND', 2)),
 ('duke', ('ND', 2))]

In [ ]:
tfidfRDD.filter(lambda token: token[0] == 'arthur').collect()

[('arthur', ('TF-I_II', 1)), ('arthur', ('ND', 1))]

In [ ]:
1 * math.log10(2/1)

0.3010299956639812

In [ ]:
tfidfRDD.saveAsTextFile("tfidf")

In [ ]:
tfidfRDD.coalesce(1,True).saveAsTextFile("tfidf2")